In [2]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys

import d2lzh_pytorch as d2l


In [3]:
def dropout(X, drop_prob):
    X = X.float()
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.randn(X.shape) < keep_prob).float()
    return mask * X / keep_prob



In [4]:
X = torch.arange(16).view(2, 8)
dropout(X, 0)


tensor([[ 0.,  1.,  2.,  0.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12.,  0., 14., 15.]])

In [5]:
num_inputs,  num_outputs, num_hiddens1,num_hiddens2 = 784, 10, 256, 256
W1 = torch.tensor(np.random.normal(0, 0.01, size=(num_inputs,num_hiddens1)), dtype=torch.float, requires_grad=True)
b1 = torch.zeros(num_hiddens1, requires_grad=True)
W2 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens1,num_hiddens2)), dtype=torch.float, requires_grad=True)
b2 = torch.zeros(num_hiddens2, requires_grad=True)
W3 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens2,num_outputs)), dtype=torch.float, requires_grad=True)
b3 = torch.zeros(num_outputs, requires_grad=True)

params = [W1,b1,W2,b2,W3,b3]

In [6]:
drop_prob1, drop_prob2 = 0.2, 0.5 
def net(X ,is_training=True):
    X = X.view(-1,num_inputs)
    H1 = (torch.matmul(X, W1) + b1).relu()
    if is_training:
        H1 = dropout(H1, drop_prob1)
    H2 = (torch.matmul(H1, W2) + b2).relu()
    if is_training:
        H2 = dropout(H2, drop_prob2) # 在第⼆层全连接后添加丢弃层
    return torch.matmul(H2, W3) + b3

In [7]:
num_epochs, lr, batch_size = 5, 100.0, 256
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs,batch_size, params, lr)

epoch 1, loss 0.0046, train acc 0.546, test acc 0.668
epoch 2, loss 0.0023, train acc 0.781, test acc 0.784
epoch 3, loss 0.0019, train acc 0.823, test acc 0.831
epoch 4, loss 0.0017, train acc 0.841, test acc 0.814
epoch 5, loss 0.0016, train acc 0.852, test acc 0.802


In [ ]:
net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs,num_hiddens1),
    nn.ReLU(),
    nn.Dropout(drop_prob1),
    nn.Linear(num_hiddens1,num_hiddens2),
    nn.ReLU(),
    nn.Dropout(drop_prob2),
    nn.Linear(num_hiddens2,10)
)
for param in params:
    nn.init.normal_(param,mean=0,std=0.01)
    

In [ ]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.5)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,optimizer)


In [15]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.5)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,optimizer)


epoch 1, loss 0.0035, train acc 0.666, test acc 0.795
epoch 2, loss 0.0021, train acc 0.804, test acc 0.830
epoch 3, loss 0.0018, train acc 0.831, test acc 0.818
epoch 4, loss 0.0017, train acc 0.844, test acc 0.830
epoch 5, loss 0.0016, train acc 0.854, test acc 0.742
